In [1]:
import numpy as np
from AbstractMesh import AbstractMesh
import utils
from metrics import aspect_ratio
from scipy.sparse import *

In [24]:
class Trimesh(AbstractMesh):
    
    def __init__(self, vertices= None, faces = None, face_normals = None, labels = None):
        
        self.face_normals         = None #npArray (Nx3)
        self.face_labels       = None #npArray (Nx1)
        self.faces            = None #npArray (Nx3)
        self.face2face         = None #npArray (Nx3?)

        if vertices and faces:
            
            self.vertices = np.array(vertices) 
            self.faces = np.array(faces)
            
            if face_normals:
                self.face_normals = np.array(face_normals)
            
            if labels:
                self.labels = np.array(labels)
        
        super(Trimesh, self).__init__()
     
    
    def show(self):
        pass
    
    
    @property
    def num_faces(self):
        return self.faces.shape[0]
    
    def add_face(self,face_id0, face_id1, face_id2):
        self.add_faces([face_id0, face_id1, face_id2])
        
    def add_faces(self, new_faces):
            
        n_vert = self.vertices.shape[0]
        new_faces = np.array(new_faces)
        self.faces = np.concatenate([self.faces, new_faces])
        
        if new_faces[(new_faces[:,0] > n_vert) | (new_faces[:,1] > n_vert) | (new_faces[:,2] > n_vert)].shape[0] > n_vert:
            raise Exception('The Id of a vertex must be lesser than the number of vertices')

        self.faces = np.concatenate([self.faces, new_faces])
        self.__compute_adjacencies()
    
    def remove_face(self,face_id):
        self.remove_faces([face_id])
        
    def remove_faces(self, face_ids):
        
        face_ids = np.array(face_ids)
        mask = np.ones(self.num_faces)
        mask[face_ids] = 0
        mask = mask.astype(np.bool)
        
        self.faces = self.faces[mask]
        self.__compute_adjacencies()
        
        
        
    
    def remove_vertex(self,vtx_id):
        
        self.remove_vertices([vtx_id])
    
    def remove_vertices(self, vtx_ids):
        
        for vtx_id in vtx_ids:
            
            self.vertices = np.delete(self.vertices, vtx_id)
            self.faces = self.faces[(self.faces[:,0] != vtx_id) & 
                                    (self.faces[:,1] != vtx_id) & 
                                    (self.faces[:,2] != vtx_id)]
            self.faces[(self.faces[:,0] > vtx_id)][0] -=1
            self.faces[(self.faces[:,1] > vtx_id)][1] -=1
            self.faces[(self.faces[:,2] > vtx_id)][2] -=1
        
        self.__compute_adjacencies()
    
    def load_from_file(self, filename):
        
        self.vertices, self.faces, self.face_normals = utils.read_obj(filename)
        self.__compute_adjacencies()
    
    def __compute_adjacencies(self):
        map_ = dict()
        adjs =  [[] for i in range(self.num_faces)]

        edges = np.c_[self.faces[:,0], self.faces[:,1], self.faces[:,1], self.faces[:,2], self.faces[:,2], self.faces[:,0]]
        edges.shape = (-1, 2)
        faces_idx = np.repeat(np.array(range(self.num_faces)), 3)


        for e, f in zip(edges, faces_idx):
            e = (e[0], e[1])

            try:
                tmp = map_[e]
            except KeyError:
                tmp = None

            if tmp is None:
                map_[(e[1], e[0])] = f
            else:
                adjs[f].append(map_[e])
                adjs[map_[e]].append(f)


        self.face2face = np.array([a.append(-1) if len(a) < 3 else a for a in adjs])
    
    
    def save_file(self, filename):
        raise NotImplementedError
        
    @property
    def simplex_centroids(self):
        
        if self._AbstractMesh__simplex_centroids is None:
            self._AbstractMesh__simplex_centroids = self.vertices[self.faces].mean(axis = 1)
        
        return self._AbstractMesh__simplex_centroids
        
        
    
    def __compute_metrics(self): 
        
        self.simplex_metrics['aspect_ratio'] = aspect_ratio(self.vertices, self.faces)
        

In [25]:
vtx, faces, face_norm = utils.read_obj('bunny.obj')

In [26]:
prova = Trimesh()

In [27]:
prova.load_from_file('dinopet.obj')

In [28]:
prova.num_faces

8996

In [191]:
a = np.array([1,2,3,4,5])

In [192]:
np.delete(a, 2)

array([1, 2, 4, 5])

In [200]:
faces[(faces[:,0] > 100) | (faces[:,1] > 100) | (faces[:,2] > 100)].shape

(28556, 3)

In [3]:
vtx[faces]

array([[[-0.435735,  0.070596,  0.449676],
        [-0.433542,  0.092247,  0.437848],
        [-0.454051,  0.088168,  0.449676]],

       [[-0.119029, -0.926244,  0.136599],
        [-0.148147, -0.937959,  0.133037],
        [-0.132589, -0.926676,  0.109416]],

       [[-0.092791, -0.972056, -0.334755],
        [-0.067982, -0.956922, -0.344027],
        [-0.06631 , -0.978755, -0.320035]],

       ...,

       [[-0.692553,  0.650989, -0.565028],
        [-0.688931,  0.626696, -0.575751],
        [-0.694702,  0.629262, -0.550228]],

       [[-0.734169,  0.465835,  0.569792],
        [-0.705903,  0.464973,  0.567883],
        [-0.722793,  0.483378,  0.551799]],

       [[-0.700132, -0.273021,  0.553894],
        [-0.719391, -0.294193,  0.54207 ],
        [-0.697134, -0.305781,  0.551193]]])

In [12]:
face_ids = np.array([0,1,2])
mask = np.ones(faces.shape[0])
mask[face_ids] = 0
mask= mask.astype(np.bool)

In [13]:
faces[mask]

array([[  144,   173,   145],
       [ 5526,  1691, 11532],
       [ 3148,  3046,  3147],
       ...,
       [ 9656,   976,  9626],
       [ 7219,  2710,  2709],
       [ 3209, 11163, 11748]])

In [15]:
faces[:7]

array([[ 6421,  6423, 13348],
       [13043,   200, 11256],
       [ 2903,  7731,  5401],
       [  144,   173,   145],
       [ 5526,  1691, 11532],
       [ 3148,  3046,  3147],
       [ 4113,  1347,  4111]])